In [1]:
import smf2021.rank2 as rank2
import smf2021.Optimiser as O
from smf.db_engine import DbEngine
import eikon as ek
# from smf.ranker import retrieve
# from smf.ranker import rank
# from smf.ranker import add_sector_info
from smf.db_engine import DbEngine
ek.set_app_key('403255e90c7647afafbfb5c0000d60ac4c8cc536')

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf

from numpy import log
from dateutil.parser import parse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import pmdarima as pm
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns
from pandas.api.types import is_numeric_dtype
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import time
import datetime as dt
db = DbEngine()

-I- Successful database connection


Optimiser 1

In [4]:
final_companies1 = ["RDN.N","HIG.N","PHM.N","HMST.OQ","BHE.N","JPM.N","MAS.N","FITB.OQ","MS.N","DHI.N","FFWM.OQ","ABBV.N", "BMY.N"]
data=ek.get_timeseries(final_companies1, fields='CLOSE', start_date='2021-01-19', end_date='2022-01-19' )
close_prices_12m = data.fillna(method="ffill")
close_prices_12m =close_prices_12m ._get_numeric_data()
close_prices_12m=close_prices_12m.astype(float)

In [5]:
min_weight, max_weight = 0.05, 0.10
print("--------------------------------------")
print(f"Weight range: [{min_weight}, {max_weight}]")
print()
mu = expected_returns.mean_historical_return(close_prices_12m)
S = risk_models.CovarianceShrinkage(close_prices_12m).ledoit_wolf()

ef = EfficientFrontier(expected_returns=mu, cov_matrix=S, weight_bounds=(min_weight, max_weight))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
print(pd.Series(cleaned_weights).sort_values(ascending=False))

--------------------------------------
Weight range: [0.05, 0.1]

Expected annual return: 22.0%
Annual volatility: 16.3%
Sharpe Ratio: 1.23
HIG.N      0.10
HMST.OQ    0.10
MAS.N      0.10
FITB.OQ    0.10
MS.N       0.10
ABBV.N     0.10
BMY.N      0.10
RDN.N      0.05
PHM.N      0.05
BHE.N      0.05
JPM.N      0.05
DHI.N      0.05
FFWM.OQ    0.05
dtype: float64


In [6]:
curr_prices = ek.get_data(final_companies1, "CF_LAST")[0]
curr_prices = curr_prices.set_index("Instrument")['CF_LAST']

In [7]:
curr_prices

Instrument
RDN.N       21.94
HIG.N       69.32
PHM.N       51.51
HMST.OQ     51.83
BHE.N       24.88
JPM.N      145.08
MAS.N       64.26
FITB.OQ     45.04
MS.N        98.86
DHI.N       87.87
FFWM.OQ     25.26
ABBV.N     131.98
BMY.N       63.54
Name: CF_LAST, dtype: Float64

In [8]:
da = DiscreteAllocation(cleaned_weights, curr_prices, total_portfolio_value=60000)
allocation, leftover = da.lp_portfolio()
print("Left Over: {}".format(leftover))

Left Over: 4.0200000000040745


In [9]:
df = pd.DataFrame()
df['Number of shares'] = pd.Series(allocation)
df['Weights'] = pd.Series(cleaned_weights).sort_values(ascending=False)
df['Current Price'] = pd.Series(curr_prices)
df

,Number of shares,Weights,Current Price
RDN.N,137,0.05,21.94
HIG.N,87,0.10,69.32
PHM.N,58,0.05,51.51
HMST.OQ,116,0.10,51.83
BHE.N,121,0.05,24.88
JPM.N,21,0.05,145.08
MAS.N,93,0.10,64.26
FITB.OQ,133,0.10,45.04
MS.N,61,0.10,98.86
DHI.N,34,0.05,87.87


In [10]:
(df['Number of shares'] * df['Current Price']).sum()
# df.to_csv("optimisation1.csv")

59995.98

Optimiser 2

In [11]:
final_companies2 = [
    "ACA.N",
    "NEM.N",
    "NSIT.OQ",
    "DIOD.OQ",
    "ETN.N",
    "FCN.N",
    "ALCO.OQ",
#     "EE.N",
    "PHM.N",
    "PCAR.OQ",
    "ADM.N",
    "CMC.N",
    "BMY.N",
    "DHI.N",
    "HMST.OQ",
    "MS.N",
    "JPM.N",
    "FFWM.OQ",
    "HIG.N",
    "RDN.N",
    "FITB.OQ",
]
len(final_companies2)

20

In [15]:
close_prices_4m=ek.get_timeseries(final_companies2, fields='CLOSE', start_date='2021-01-19', end_date='2022-01-19')
close_prices_4m = close_prices_4m.loc[:"2022-01-19":,final_companies2].last("90D").fillna(method="ffill")
close_prices_4m=close_prices_4m.astype(float)
close_prices_4m.shape

(61, 20)

In [13]:
close_prices_4m

CLOSE,ACA.N,NEM.N,NSIT.OQ,DIOD.OQ,ETN.N,FCN.N,ALCO.OQ,PHM.N,PCAR.OQ,ADM.N,CMC.N,BMY.N,DHI.N,HMST.OQ,MS.N,JPM.N,FFWM.OQ,HIG.N,RDN.N,FITB.OQ
Date,,,,,,,,,,,,,,,,,,,,
2021-10-22,52.92,57.61,94.26,92.11,162.38,142.50,35.35,49.72,87.00,66.22,31.51,57.60,88.59,43.62,101.80,171.78,26.81,74.10,24.33,45.49
2021-10-25,53.54,57.91,95.31,94.09,162.51,144.47,35.49,49.86,87.23,66.63,32.89,57.81,88.57,44.09,101.91,170.94,27.00,73.80,24.37,45.58
2021-10-26,52.72,57.70,94.11,93.89,161.34,145.98,35.60,48.68,86.88,65.36,32.29,58.24,88.70,45.19,103.47,171.40,27.38,72.94,24.21,44.79
2021-10-27,51.59,57.54,92.34,93.54,160.12,142.43,35.57,48.75,85.77,64.53,31.34,56.49,87.77,46.55,101.87,167.83,26.19,72.81,23.84,43.24
2021-10-28,53.11,54.92,93.47,96.05,164.96,144.10,35.57,48.66,89.40,64.84,31.98,57.13,89.36,47.12,102.50,170.36,26.35,73.78,23.88,43.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-12,49.95,61.52,103.61,99.27,171.46,151.32,37.30,55.02,92.54,70.42,36.97,64.82,99.43,55.87,103.05,168.44,27.22,72.32,23.45,49.20
2022-01-13,52.25,61.32,103.02,96.91,171.18,150.21,37.50,56.93,95.32,70.90,37.85,64.60,99.76,56.57,102.55,168.23,27.50,72.41,23.52,49.54
2022-01-14,52.41,61.22,102.77,97.92,170.92,151.30,37.42,57.09,96.50,71.55,37.92,64.93,98.53,56.63,98.88,157.89,27.56,72.45,23.59,50.45


In [16]:
min_weight, max_weight = 0.02, 0.1
print("--------------------------------------")
print(f"Weight range: [{min_weight}, {max_weight}]")
print(close_prices_4m.shape)
print()
mu = expected_returns.mean_historical_return(close_prices_4m)
S = risk_models.CovarianceShrinkage(close_prices_4m).ledoit_wolf()

ef = EfficientFrontier(expected_returns=mu, cov_matrix=S, weight_bounds=(min_weight, max_weight))
raw_weights = ef.max_sharpe(risk_free_rate=0.00)
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
print(pd.Series(cleaned_weights).sort_values(ascending=False))

--------------------------------------
Weight range: [0.02, 0.1]
(61, 20)

Expected annual return: 47.3%
Annual volatility: 13.2%
Sharpe Ratio: 3.59
CMC.N      0.10
HMST.OQ    0.10
FCN.N      0.10
PCAR.OQ    0.10
ADM.N      0.10
NEM.N      0.10
BMY.N      0.10
ALCO.OQ    0.06
RDN.N      0.02
HIG.N      0.02
FFWM.OQ    0.02
JPM.N      0.02
MS.N       0.02
ACA.N      0.02
DHI.N      0.02
PHM.N      0.02
ETN.N      0.02
DIOD.OQ    0.02
NSIT.OQ    0.02
FITB.OQ    0.02
dtype: float64


C:\ProgramData\Anaconda3\lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:438: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


In [31]:
curr_prices = ek.get_data(final_companies2, "CF_LAST")[0]
# curr_prices['Instrument'] = curr_prices['Instrument'].apply(lambda x: x.replace(".", "_"))
curr_prices = curr_prices.set_index("Instrument")['CF_LAST']
da = DiscreteAllocation(cleaned_weights, curr_prices, total_portfolio_value=100000)
allocation, leftover = da.lp_portfolio()
print("Left Over: {}".format(leftover))

Left Over: 15.389999999984866


In [33]:
df = pd.DataFrame()
df['Number of shares'] = pd.Series(allocation)
df['Weights'] = pd.Series(cleaned_weights).sort_values(ascending=False)
df['Current Price'] = pd.Series(curr_prices)
df.sort_values("Weights", ascending=False)

,Number of shares,Weights,Current Price
CMC.N,297,0.10,33.68
HMST.OQ,193,0.10,51.83
FCN.N,67,0.10,148.83
PCAR.OQ,109,0.10,91.56
ADM.N,145,0.10,68.94
NEM.N,159,0.10,63.09
BMY.N,158,0.10,63.54
ALCO.OQ,167,0.06,35.93
RDN.N,91,0.02,21.94
HIG.N,29,0.02,69.32


In [34]:
(df['Number of shares'] * df['Current Price']).sum()

99984.60999999997

In [ ]:
# df.to_csv("optimisation2.csv")

Rebalance Code

In [70]:
# Industrials
ind_fields = ['TR.GICSSector','TR.PE',"TR.PriceToBVPerShare",
              "TR.EPSMean", "TR.EVToSales", "TR.EVToEBIT",
              "TR.PtoEBTMeanEst", "TR.FwdPEG"]


# Comms
comms_fields = ['TR.GICSSector','TR.ROATotalAssetsPercent',
                'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                'TR.EVToEBITDA',
                'TR.ROICActValue', 'TR.TimesInterestEarned', 'TR.FCFMeanYield',
               ]

# Healthcare
health_fields = ['TR.GICSSector','TR.PE',  'TR.EVToEBITDA',
                 'TR.EVToSales', 'TR.ROICActValue', 'TR.FCFMeanYield',
'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                 "TR.PriceToBVPerShare"]

cons_fields = ['TR.GICSSector','TR.PE', 'TR.ROATotalAssetsPercent', 
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.EVToEBITDA', 'TR.EVToSales']

# fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", "TR.EPSMean",
#           "TR.EVToSales", "TR.EVToEBIT",
#           "TR.PtoEBTMeanEst", "TR.FwdPEG"]  
# Tech
tech_fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", 
               'TR.ROICActValue', 'TR.ReinvestmentRatePct',
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.FCFMeanYield']

In [71]:
ind_ranker = rank2.ranker(ind_fields)
comms_ranker = rank2.ranker(comms_fields)
health_ranker = rank2.ranker(health_fields)
cons_ranker = rank2.ranker(cons_fields)
tech_ranker = rank2.ranker(tech_fields)

In [72]:
all_comms = comms_ranker.get_data()
all_ind = ind_ranker.get_data()
all_health = health_ranker.get_data()
all_cons = cons_ranker.get_data()
all_tech = tech_ranker.get_data()
all_tech = tech_ranker.get_data()

In [73]:
ind_data = ind_ranker.industry_data('Industrials')
comms_data = comms_ranker.industry_data('Communication Services')
cons_data = cons_ranker.industry_data('Consumer Staples')
health_data = health_ranker.industry_data('Health Care')
tech_data = health_ranker.industry_data('Information Technology')

In [139]:
health_data

,Instrument,GICS Sector Name,P/E (Daily Time Series Ratio),Enterprise Value To EBITDA (Daily Time Series Ratio),Enterprise Value To Sales (Daily Time Series Ratio),Return On Invested Capital - Actual,Free Cash Flow Yield %,ROE Common Equity %,Price To Book Value Per Share (Daily Time Series Ratio)
28,BIO.N,Health Care,2.657075,21.248814,5.433339,3.54,2.478021,48.689066,1.142844
38,SYK.N,Health Care,51.199678,25.292363,6.362162,13.29,2.946387,12.351782,6.775659
40,TECH.OQ,Health Care,84.793721,41.152576,14.725533,17.44,2.247265,9.532353,8.794344
44,BSX.N,Health Care,55.951249,27.590769,5.960761,<NA>,2.661551,-0.78759,3.718917
57,UNH.N,Health Care,25.501323,16.912922,1.592101,20.58,5.178319,24.598679,5.787908
...,...,...,...,...,...,...,...,...,...
408,WST.N,Health Care,45.695344,33.233291,10.000977,22.86,1.514138,20.200134,12.375235
442,ABBV.N,Health Care,31.415582,11.354872,5.470896,21.5,9.108245,185.807504,17.218831
462,ABT.N,Health Care,31.011095,19.069261,5.469995,<NA>,3.389390,14.081288,6.463992
483,CI.N,Health Care,9.92409,9.836664,0.641022,<NA>,8.811113,17.683647,1.651234


In [74]:
# True => lower is better.
# ROIC < WACC => remove from list.
# Interest Coverage Ratio > 1.5

asc_ind = {'P/E (Daily Time Series Ratio)': True,
           'Price To Book Value Per Share (Daily Time Series Ratio)': True,
           'Earnings Per Share - Mean': False, 
           'Enterprise Value To Sales (Daily Time Series Ratio)': True,
           'Enterprise Value To EBIT (Daily Time Series Ratio)': False, 
           'Price / EBITDA (Mean Estimate)': True, 
           'Forward P/E/G (Daily Time Series Ratio)': True}
             
asc_comms = {'ROA Total Assets, Percent': False,
             'ROE Common Equity %': False,
             'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
             'Return On Invested Capital - Actual': False,
             'Interest Coverage Ratio': False,
             'Free Cash Flow Yield %': False}



asc_tech = {'ROA Total Assets, Percent': False,
            'ROE Common Equity %': False,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Return On Invested Capital - Actual': False,
            'Interest Coverage Ratio': False,
            'Price / Sales Ratio - Actual': True,
            'Hist EBITDA (Net Cap Ex) Interest Cover, %': False,
            'Free Cash Flow Yield %': False}

asc_health = {'P/E (Daily Time Series Ratio)': True,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False,
            'ROE Common Equity %': False,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True}

asc_cons = {'P/E (Daily Time Series Ratio)': True,
                'ROA Total Assets, Percent': False,
                'ROE Common Equity %': False,
                'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
                'Enterprise Value To Sales (Daily Time Series Ratio)': True
                }

In [75]:
ind_z_scores = ind_ranker.z_scores(ind_data, asc_ind)
health_z_scores = health_ranker.z_scores(health_data, asc_health)
comms_z_scores = comms_ranker.z_scores(comms_data, asc_comms)
# tech_z_scores = tech_ranker.z_scores(tech_data, asc_comms)
cons_z_scores = cons_ranker.z_scores(cons_data, asc_cons)


In [78]:
cons_list = cons_z_scores.head(10)['Instrument'].tolist()
ind_list = ind_z_scores.head(10)['Instrument'].tolist()
health_list = health_z_scores.head(10)['Instrument'].tolist()
comms_list = comms_z_scores.head(10)["Instrument"].tolist()

In [83]:
# stocks_list = comms_list+ind_list+health_list
# close_prices = O.get_close_data(stocks_list.copy())
stocks_list = ind_list
close_prices = O.get_close_data(stocks_list.copy())
close_prices=close_prices.astype(float)

In [67]:
# # stocks_list = comms_list+ind_list+health_list+cons_list+tech_list
# stocks_list = comms_list+ind_list+health_list+cons_list+tech_list

# close_prices = O.get_close_data(stocks_list.copy())
# close_prices=close_prices.astype(float)

In [84]:
cleaned_weights = O.optimiser(close_prices,"Minimum",lower_weight=0.01,upper_weight=0.1)
O.allocate(stocks_list,cleaned_weights)

Optimizer Error, Try changing weights


dp_populate_and_update

In [99]:
price=ek.get_timeseries('AAPL.N', fields='CLOSE', start_date='2021-01-19', end_date='2022-01-19')

In [121]:
time_ = time.time()
tickers_list= ["AAPL.N", "RDN.N","HIG.N","PHM.N","HMST.OQ","BHE.N","JPM.N","MAS.N","FITB.OQ","MS.N","DHI.N","FFWM.OQ","ABBV.N", "BMY.N"]
start = dt.date(2020, 6, 30).strftime("%Y-%m-%d")
end = dt.datetime.today().strftime("%Y-%m-%d")
eikon_data = ek.get_data(tickers_list, ["TR.TRBCEconomicSector","TR.TRBCBusinessSector", "TR.TRBCIndustryGroup"])
sectors_df = pd.DataFrame(data=eikon_data[0])
sectors_df.columns = ["ric", "economic_sector", "business_sector", "industry_group"]
sectors_df = sectors_df.set_index("ric")

In [123]:
volatilities_days = [2, 5, 10, 20, 25, 30, 40, 50, 60, 80, 90, 100, 120, 150, 160, 180, 200, 240, 250, 260]
volatilities_fields = list(map(lambda num:f"TR.Volatility{str(num)}D", volatilities_days))
eikon_data, err = ek.get_data(tickers_list, volatilities_fields)
volatilities_df = pd.DataFrame(eikon_data)
volatilities_df.columns = ["ric"] + list(map(lambda num:f"{str(num)}_day_volatility", volatilities_days))
volatilities_df = volatilities_df.set_index("ric")

today = dt.datetime.now().strftime("%Y_%m_%d")
volatilities_table_name = f"volatilities_as_at_{today}"

In [124]:
time_ = time.time()
start = dt.date(2021, 6, 30).strftime("%Y-%m-%d")
end = dt.datetime.today().strftime("%Y-%m-%d")
problem_stocks = []


In [182]:
stonks = []

for (count, ticker) in reversed(list(enumerate(tickers_list))):
    try:
        # Closing prices
        ek_closing_prices = ek.get_timeseries(ticker, ["Close"], start_date=start, end_date=end)
        ticker_info_df = pd.DataFrame(ek_closing_prices)
        ticker_info_df.columns = ["close_price"]

        # Price-earnings
        date_field = f"TR.HistPE(SDate={start},EDate={end},Period=FI0,Frq=FI).date"
        field = f"TR.HistPE(SDate={start},EDate={end},Period=FI0,Frq=FI)"
        eik_data, err = ek.get_data(ticker, [date_field, field])
        data_df = pd.DataFrame(eik_data, columns=["Date", "Historic P/E"]).set_index("Date")
        if data_df["Historic P/E"].isna().all():
            ticker_info_df["price_earnings"] = np.nan
        else:
            data_df.columns = ["price_earnings"]
            data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
            ticker_info_df = ticker_info_df.join(data_df)
            ticker_info_df = ticker_info_df.fillna(method="ffill")

        # Price-to-book
        date_field = f"TR.PriceToBVPerShare(SDate={start},EDate={end},Period=FI0,Frq=FI).date"
        field = f"TR.PriceToBVPerShare(SDate={start},EDate={end},Period=FI0,Frq=FI)"
        eik_data, err = ek.get_data(ticker, [date_field, field])
        data_df = pd.DataFrame(eik_data, columns=["Date", "Price To Book Value Per Share (Daily Time Series Ratio)"]).set_index("Date")
        if data_df["Price To Book Value Per Share (Daily Time Series Ratio)"].isna().all():
            ticker_info_df["price_to_book"] = np.nan
        else:
            data_df.columns = ["price_to_book"]
            data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
            ticker_info_df = ticker_info_df.join(data_df)
            ticker_info_df["price_to_book"] = ticker_info_df["price_to_book"].fillna(method="ffill")

        # Price-to-free-cashflow
        date_field = f"TR.HistPricetoFreeOperatingCashFlowperShareAvgDilutedSharesOut(SDate={start},EDate={end},Period=FI0,Frq=FI).date"
        field = f"TR.HistPricetoFreeOperatingCashFlowperShareAvgDilutedSharesOut(SDate={start},EDate={end},Period=FI0,Frq=FI)"
        eik_data, err = ek.get_data(ticker, [date_field, field])
        data_df = pd.DataFrame(eik_data, columns=["Date", "Historic Price/FOCF/Shr (dil.)"]).set_index("Date")
        if data_df["Historic Price/FOCF/Shr (dil.)"].isna().all():
            ticker_info_df["price_to_fcf"] = np.nan
        else:
            data_df.columns = ["price_to_fcf"]
            data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
            ticker_info_df = ticker_info_df.join(data_df)
            ticker_info_df["price_to_fcf"] = ticker_info_df["price_to_fcf"].fillna(method="ffill")

        # EV-EBITDA
        date_field = f"TR.HistEnterpriseValueEBITDA(SDate={start},EDate={end},Period=FY0,Frq=FY).date"
        field = f"TR.HistEnterpriseValueEBITDA(SDate={start},EDate={end},Period=FY0,Frq=FY)"
        eik_data, err = ek.get_data(ticker, [date_field, field])
        data_df = pd.DataFrame(eik_data, columns=["Date", "Historic EV/EBITDA, FY"]).set_index("Date")
        if data_df["Historic EV/EBITDA, FY"].isna().all():
            ticker_info_df["ev_ebitda"] = np.nan
        else:
            data_df.columns = ["ev_ebitda"]
            data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
            ticker_info_df = ticker_info_df.join(data_df)
            ticker_info_df["ev_ebitda"] = ticker_info_df["ev_ebitda"].fillna(method="ffill")

#         # Current ratio
#         date_field = f"TR.CurrentRatio(SDate={start},EDate={end},Frq=Y).date"
#         field = f"TR.CurrentRatio(SDate={start},EDate={end},Frq=Y)"
#         eik_data, err = ek.get_data(ticker, [date_field, field])
#         data_df = pd.DataFrame(eik_data, columns=["Date", "Current Ratio"]).set_index("Date")
#         data_df.columns = ["current_ratio"]
#         data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
#         ticker_info_df = ticker_info_df.join(data_df)
#         ticker_info_df["current_ratio"] = ticker_info_df["current_ratio"].fillna(method="ffill")

#         # Quick Ratio
#         date_field = f"TR.QuickRatio(SDate={start},EDate={end},Frq=Y).date"
#         field = f"TR.QuickRatio(SDate={start},EDate={end},Frq=Y)"
#         eik_data, err = ek.get_data(ticker, [date_field, field])
#         data_df = pd.DataFrame(eik_data, columns=["Date", "Quick Ratio"]).set_index("Date")
#         data_df.columns = ["quick_ratio"]
#         data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
#         ticker_info_df = ticker_info_df.join(data_df)
#         ticker_info_df["quick_ratio"] = ticker_info_df["quick_ratio"].fillna(method="ffill")

#         # Dollar Turnover
#         date_field = f"TR.TURNOVER(SDate={start},EDate={end},Frq=D).date"
#         field = f"TR.TURNOVER(SDate={start},EDate={end},Frq=D)"
#         eik_data, err = ek.get_data(ticker, [date_field, field])
#         data_df = pd.DataFrame(eik_data, columns=["Date", "Turnover"]).set_index("Date")
#         data_df.columns = ["dollar_turnover"]
#         data_df.index = data_df.index.str[0:10]#pd.to_datetime(data_df.index, format="%Y-%m-%dT%H:%M:%SZ")
#         ticker_info_df = ticker_info_df.join(data_df)
        
        # Because duplicate rows arise, we can't create SQL primary keys until we get rid of these duplicates...
          # Because duplicate rows arise, we can't create SQL primary keys until we get rid of these duplicates...
        ticker_info_df = ticker_info_df.loc[~ticker_info_df.index.duplicated(keep='first')]
        
        # Since '.' has a specific function in SQL, we use '_' instead.
        
#         db.append_df_as_row(ticker_info_df, ticker.replace(".","_"), if_exists="replace", index=True) 
#         db.set_primary_key(ticker.replace(".","_"), "Date")
        stonks.append(ticker_info_df) 
        
        print(f"-I- Successfully stored table for ({count}) {ticker}. {count-1} more to go.")

    except Exception as e:
        problem_stocks.append(ticker)
        print(f"-W- Could not store table for ({count}) {ticker}. Error message: {str(e)}")


print("\n",f"Whole thing took {time.time() - time_} seconds")

-I- Successfully stored table for (13) BMY.N. 12 more to go.
-I- Successfully stored table for (12) ABBV.N. 11 more to go.
-I- Successfully stored table for (11) FFWM.OQ. 10 more to go.
-I- Successfully stored table for (10) DHI.N. 9 more to go.
-I- Successfully stored table for (9) MS.N. 8 more to go.
-I- Successfully stored table for (8) FITB.OQ. 7 more to go.
-I- Successfully stored table for (7) MAS.N. 6 more to go.
-I- Successfully stored table for (6) JPM.N. 5 more to go.
-I- Successfully stored table for (5) BHE.N. 4 more to go.
-I- Successfully stored table for (4) HMST.OQ. 3 more to go.
-I- Successfully stored table for (3) PHM.N. 2 more to go.
-I- Successfully stored table for (2) HIG.N. 1 more to go.
-I- Successfully stored table for (1) RDN.N. 0 more to go.
-I- Successfully stored table for (0) AAPL.N. -1 more to go.

 Whole thing took 5079.342501401901 seconds


In [102]:
problem_stocks = []

for (count, ticker) in reversed(list(enumerate(tickers_list))):
    try:
        # Closing prices
        ek_closing_prices = ek.get_timeseries(ticker, ["Close"], start_date=start, end_date=end)
        ticker_info_df = pd.DataFrame(ek_closing_prices)
        ticker_info_df.columns = ["close_price"]
        ticker_info_df = ticker_info_df.loc[~ticker_info_df.index.duplicated(keep='first')]
        print(f"-I- Successfully stored table for ({count}) {ticker}. {count-1} more to go.")

    except Exception as e:
        problem_stocks.append(ticker)
        print(f"-W- Could not store table for ({count}) {ticker}. Error message: {str(e)}")


print("\n",f"Whole thing took {time.time() - time_} seconds")

-I- Successfully stored table for (13) BMY.N. 12 more to go.
-I- Successfully stored table for (12) ABBV.N. 11 more to go.
-I- Successfully stored table for (11) FFWM.OQ. 10 more to go.
-I- Successfully stored table for (10) DHI.N. 9 more to go.
-I- Successfully stored table for (9) MS.N. 8 more to go.
-I- Successfully stored table for (8) FITB.OQ. 7 more to go.
-I- Successfully stored table for (7) MAS.N. 6 more to go.
-I- Successfully stored table for (6) JPM.N. 5 more to go.
-I- Successfully stored table for (5) BHE.N. 4 more to go.
-I- Successfully stored table for (4) HMST.OQ. 3 more to go.
-I- Successfully stored table for (3) PHM.N. 2 more to go.
-I- Successfully stored table for (2) HIG.N. 1 more to go.
-I- Successfully stored table for (1) RDN.N. 0 more to go.
-I- Successfully stored table for (0) AAPL.N. -1 more to go.

 Whole thing took 18.260700464248657 seconds


In [104]:
def volatility(period, as_at, price_table):
    business_days_per_year = 224.5819722273368 # This number was obtained by comparing with the annualized historical
                                               # volatility given by Eikon on the DIB for TR.Volatility60D, etc.
    as_at = pd.Timestamp(as_at)
    # Calculate log-returns
    returns = price_table.dropna().pct_change() + 1
    log_returns = returns.transform(np.log)
    
    # Calculate standard deviation of log-returns (as a percentage)
    log_returns = pd.DataFrame(log_returns.loc[:as_at,:].iloc[-60:])#* np.sqrt(business_days_per_year))
    volatilities = log_returns.std() * 100
    
    # Rename headings/column name
    volatilities.index.name = "ric"
    volatilities.columns = [f"{period}_day_volatility"]
    
    return(log_returns)

In [187]:
period = 60
as_at = "2021-06-30"
new = volatility(period, as_at, price)

In [192]:
tickers_list = ["AAPL.N", "RDN.N","HIG.N","PHM.N","HMST.OQ","BHE.N","JPM.N","MAS.N","FITB.OQ","MS.N","DHI.N","FFWM.OQ","ABBV.N", "BMY.N"]
volatilities_days = [2, 5, 10, 20, 25, 30, 40, 50, 60, 80, 90, 100, 120, 150, 160, 180, 200, 240, 250, 260]
volatilities_fields = list(map(lambda num:f"TR.Volatility{str(num)}D", volatilities_days))
eikon_data, err = ek.get_data(tickers_list, volatilities_fields)
volatilities_df = pd.DataFrame(eikon_data)
volatilities_df.columns = ["ric"] + list(map(lambda num:f"{str(num)}_day_volatility", volatilities_days))
volatilities_df = volatilities_df.set_index("ric")

today = dt.datetime.now().strftime("%Y_%m_%d")
volatilities_table_name = f"volatilities_as_at_{today}"
# db.append_df_as_row(volatilities_df, volatilities_table_name, if_exists="replace", index=True)
# db.set_primary_key(volatilities_table_name, "ric")

In [214]:
volatilities=volatilities_df
price = ek.get_data(tickers_list,["Close"])
price=ek.get_timeseries('AAPL.N', fields='CLOSE', start_date='2021-01-19', end_date='2022-01-19')

In [215]:
period = 60
as_at = "2021-06-30"
new = volatility(period, as_at, price)

In [216]:
test = pd.DataFrame(index=new.index)
test["new"] = new
# test["old"] = volatilities.loc[:,"60_day_volatility"]
# test["old"]

In [217]:
test["new"]

Date
2021-04-07     0.01322
2021-04-08     0.01901
2021-04-09    0.019107
2021-04-12   -0.012342
2021-04-13    0.023566
2021-04-14   -0.017073
2021-04-15    0.017445
2021-04-16   -0.002383
2021-04-19    0.005428
2021-04-20   -0.013513
2021-04-21    0.003452
2021-04-22   -0.011375
2021-04-23     0.01668
2021-04-26    0.003941
2021-04-27   -0.004016
2021-04-28   -0.004256
2021-04-29   -0.001273
2021-04-30   -0.014947
2021-05-03    0.008331
2021-05-04   -0.035234
2021-05-05    0.001405
2021-05-06    0.011558
2021-05-07    0.004079
2021-05-10   -0.026068
2021-05-11   -0.007836
2021-05-12   -0.024615
2021-05-13    0.017118
2021-05-14     0.01966
2021-05-17   -0.009385
2021-05-18   -0.010835
2021-05-19   -0.001363
2021-05-20    0.020405
2021-05-21   -0.014168
2021-05-24    0.013382
2021-05-25   -0.001811
2021-05-26   -0.000552
2021-05-27   -0.012216
2021-05-28   -0.004479
2021-06-01   -0.003936
2021-06-02    0.006738
2021-06-03   -0.012226
2021-06-04    0.018203
2021-06-07    0.000794
2021-0

In [220]:
v1=volatilities.loc[:,"60_day_volatility"]
test["old"]=np.log(v1["AAPL.N"])

In [221]:
test

,new,old
Date,,
2021-04-07,0.01322,3.26224
2021-04-08,0.01901,3.26224
2021-04-09,0.019107,3.26224
2021-04-12,-0.012342,3.26224
2021-04-13,0.023566,3.26224
2021-04-14,-0.017073,3.26224
2021-04-15,0.017445,3.26224
2021-04-16,-0.002383,3.26224
2021-04-19,0.005428,3.26224


In [222]:
# test = pd.DataFrame(index=new.index)
# test["new"] = new
# # test["old"] = volatilities.loc[:,"60_day_volatility"]
# test["old"] = v1['AAPL.N']
test["ratio"] = (test["old"] / test["new"])**2
test["ratio"].describe()

count        60.000000
unique       60.000000
top       60891.284066
freq          1.000000
Name: ratio, dtype: float64

Portfolio optimisation markowitz

In [2]:
cons_list = cons_z_scores.head(10)['Instrument'].tolist()
ind_list = ind_z_scores.head(10)['Instrument'].tolist()
health_list = health_z_scores.head(10)['Instrument'].tolist()
comms_list = comms_z_scores.head(10)["Instrument"].tolist()
tech_list = tech_z_scores.head(10)["Instrument"].tolist()
stocks_list = comms_list+ind_list+health_list+cons_list+tech_list
close_prices = O.get_close_data(stocks_list.copy())
close_prices=close_prices.astype(float)

In [ ]:
returns_portfolio=close_prices.pct_change() 
returns_portfolio.head(n=5)
weight_portfolio= np.random.random(len(returns_portfolio.columns))
portfolio_return = returns_portfolio.dot(weight_portfolio)
portfolio=portfolio_return

In [ ]:
variance_matrix=returns_portfolio.cov()*252
variance_matrix

In [ ]:
portfolio_variance=np.transpose(weight_portfolio)@variance_matrix@weight_portfolio
portfolio_volatility=np.sqrt(portfolio_variance)
print(portfolio_variance)
print(portfolio_volatility)

In [ ]:
port_returns=[]
port_volatility=[]
port_weights=[]

num_assets=len(returns_portfolio.columns)
num_portfolios = 1000

individual_rets = returns_portfolio.resample('Y').last().pct_change().mean()

In [ ]:
for port in range(num_portfolios):
    weights=np.random.random(num_assets)
    weights=weights/np.sum(weights)
    port_weights.append(weights)
    returns=np.dot(weights, individual_rets)
    port_returns.append(returns)
    
    var=variance_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()
    sd=np.sqrt(var)
    ann_sd=sd*np.sqrt(250)
    port_volatility.append(ann_sd)

In [ ]:
data= {'Returns':port_returns, 'Volatility':port_volatility}

for counter, symbol in enumerate(portfolio.columns.tolist()):
    data[symbol+'weight']=[w[counter] for w in port_weights]
